# Perfil de comércio

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=12


### Perfil de comércio

Escolher país e anos de interesse.

In [30]:
country_of_interest = comtrade.m49_mozambique
years_of_interest = comtrade.year_range(2020, 2022)
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Mozambique 2020,2021


### Global exports and imports

In [31]:
cols = ['reporterDesc','partnerDesc','flowDesc','primaryValueFormated']
global_trade = comtrade.get_global_stats(country_of_interest, years_of_interest)
global_trade[cols].sort_index()


reporterDesc partnerDesc flowDesc primaryValueFormated
refYear flowCode                                                       
2020    M          Mozambique       World   Import     6,437,596,781.25
        X          Mozambique       World   Export     3,460,032,908.61
2021    M          Mozambique       World   Import     8,622,696,043.56
        X          Mozambique       World   Export     5,111,685,900.99

## Principais parceiros comerciais
### Principais destinos das exportações

In [6]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 500)

top_partners = comtrade.top_partners(country_of_interest,years_of_interest,flowCode='X', 
                                    global_stats=global_trade,rank_filter=10)
top_partners.sort_values(by=['refYear','flowCode','rank'], ascending=[True,True,True]).style.format({'perc': '{:,.2%}'.format})

## Principais produtos exportados

In [7]:
export_products, df = comtrade.top_commodities(country_of_interest,0,years_of_interest, flowCode='X',
                                           motCode=0,rank_filter=5,
                                           pco_cols=['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
                                                     'flowCode','primaryValue','primaryValueFormated'],
                                           global_stats=global_trade)


In [8]:
cols = ['cmdCode','cmdDesc','primaryValueFormated','perc']
export_products[cols].style.format({'perc': '{:,.2%}'.format})

### Principais produtos por parceiros principais

In [9]:
export_products, df = comtrade.top_commodities(country_of_interest,None,years_of_interest, flowCode='X',
                                           motCode=0, 
                                             partner_first=True,
                                           global_stats=global_trade, 
                                           rank_filter=3)
df = df[df.partnerCode!=0]

In [10]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 2500)

export_products[['cmdDesc','primaryValue','perc']].reset_index().head(10).style.format({'perc': '{:,.2%}'.format})

,reporterDesc,partnerDesc,flowCode,refYear,rank,cmdDesc,primaryValue,perc
0,Mozambique,Afghanistan,X,2021,1,Plastics and articles thereof,8079.000000,0.00%
1,Mozambique,Afghanistan,X,2021,2,"Fruit and nuts, edible; peel of citrus fruit or melons",6186.400000,0.00%
2,Mozambique,Angola,X,2021,1,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",308127.750000,0.01%
3,Mozambique,Angola,X,2021,2,"Printed books, newspapers, pictures and other products of the printing industry; manuscripts, typescripts and plans",272032.850000,0.01%
4,Mozambique,Angola,X,2021,3,Plastics and articles thereof,122923.620000,0.00%
5,Mozambique,Argentina,X,2021,1,Animal originated products; not elsewhere specified or included,1460.000000,0.00%
6,Mozambique,Argentina,X,2021,2,"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings",505.190000,0.00%
7,Mozambique,"Asia, not elsewhere specified",X,2021,1,Aluminium and articles thereof,144564.700000,0.00%
8,Mozambique,"Asia, not elsewhere specified",X,2021,2,"Trees and other plants, live; bulbs, roots and the like; cut flowers and ornamental foliage",10834.540000,0.00%
9,Mozambique,"Asia, not elsewhere specified",X,2021,3,"Ores, slag and ash",5544.700000,0.00%


In [11]:
# label for the percentage of a commodity in the total trade of a country
cmd_in_country = 'perc_cmd_for_partner'
# label for the percentage of a country in the total trade of a commodity
country_in_cmd = 'perc_partner_for_cmd'
# Total value of commodities per HS code
df['sum_cmd'] = df.groupby(['reporterDesc','refYear','flowCode','cmdCode'])['primaryValue'].transform('sum')  
# Total value of commodities per partner
df['sum_partner'] = df.groupby(['reporterDesc','refYear','flowCode','partnerCode'])['primaryValue'].transform('sum')
# rank of commodity per HS code
df['rank_cmd'] = df.groupby(['reporterDesc','refYear','flowCode'])['sum_cmd'].rank(ascending=False,method='dense')
df['rank_cmd'] = df['rank_cmd'].astype(int)
# value of trade as percentage of total same HS code
df[country_in_cmd] = df['primaryValue']/df['sum_cmd']
# value of trade as percentage of total same partner
df[cmd_in_country] = df['primaryValue']/df['sum_partner']

# rank of partner per HS code
df['rank_cmd_partner'] = df.groupby(['reporterDesc','refYear','flowCode','cmdCode'])['primaryValue'].rank(ascending=False,method='dense')
df['rank_cmd_partner'] = df['rank_cmd_partner'].astype(int)
# rank of commodity per partner
df['rank_partner_cmd'] = df.groupby(['reporterDesc','refYear','flowCode','partnerCode'])['primaryValue'].rank(ascending=False,method='dense')
df['rank_partner_cmd'] = df['rank_partner_cmd'].astype(int)

# global rank of partner
df['rank_partner'] = df.groupby(['reporterDesc','refYear','flowCode'])['sum_partner'].rank(ascending=False,method='dense')
df['rank_partner'] = df['rank_partner'].astype(int)



### Principais produtos e principais parceiros

In [12]:
show_cols = ['reporterDesc','refYear','flowDesc','rank_cmd','cmdDesc','rank_cmd_partner','rank_partner','partnerDesc','primaryValueFormated',cmd_in_country,country_in_cmd]
sort_list = ['refYear','flowCode','rank_cmd','rank_cmd_partner']
rank_cut = 5
rank_filter = (df['rank_cmd'] <= rank_cut) & (df['rank_cmd_partner'] <= rank_cut)

df[rank_filter].sort_values(by=sort_list)[show_cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_cmd_partner,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
1579,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1,1,South Africa,"623,155,865.72",72.86%,28.31%
1464,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,2,India,"578,305,515.42",72.00%,26.27%
882,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,7,Republic of Korea,"190,353,794.47",98.47%,8.65%
255,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,4,China,"139,905,599.80",28.59%,6.36%
1158,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",5,3,Netherlands,"135,251,228.91",24.08%,6.15%
1165,Mozambique,2021,Export,2,Aluminium and articles thereof,1,3,Netherlands,"408,356,074.50",72.72%,36.19%
2033,Mozambique,2021,Export,2,Aluminium and articles thereof,2,5,United Kingdom,"354,618,126.67",95.26%,31.43%
1537,Mozambique,2021,Export,2,Aluminium and articles thereof,3,8,Singapore,"109,367,719.74",72.65%,9.69%
758,Mozambique,2021,Export,2,Aluminium and articles thereof,4,9,Italy,"95,211,869.74",70.59%,8.44%
1716,Mozambique,2021,Export,2,Aluminium and articles thereof,5,10,Spain,"37,164,627.70",35.19%,3.29%


## Principais parceiros comerciais e principais produtos

In [32]:
show_cols = ['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','rank_partner_cmd','rank_cmd','cmdDesc','primaryValueFormated',cmd_in_country,country_in_cmd]
sort_list = ['refYear','flowCode','rank_partner','rank_partner_cmd']
rank_cut = 5
rank_filter = (df['rank_partner'] <= rank_cut) & (df['rank_partner_cmd'] <= rank_cut)

df[rank_filter].sort_values(by=sort_list)[show_cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_partner_cmd,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
1579,Mozambique,2021,Export,1,South Africa,1,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","623,155,865.72",72.86%,28.31%
1563,Mozambique,2021,Export,1,South Africa,2,6,"Fruit and nuts, edible; peel of citrus fruit or melons","53,160,569.62",6.22%,44.81%
1613,Mozambique,2021,Export,1,South Africa,3,14,"Feathers and down, prepared; and articles made of feather or of down; artificial flowers; articles of human hair","41,737,465.08",4.88%,98.17%
1598,Mozambique,2021,Export,1,South Africa,4,21,"Printed books, newspapers, pictures and other products of the printing industry; manuscripts, typescripts and plans","19,023,600.07",2.22%,97.16%
1622,Mozambique,2021,Export,1,South Africa,5,2,Aluminium and articles thereof,"15,146,373.59",1.77%,1.34%
1464,Mozambique,2021,Export,2,India,1,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","578,305,515.42",72.00%,26.27%
1448,Mozambique,2021,Export,2,India,2,4,Vegetables and certain roots and tubers; edible,"143,294,446.93",17.84%,92.41%
1452,Mozambique,2021,Export,2,India,3,7,"Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder","27,313,036.97",3.40%,23.50%
1463,Mozambique,2021,Export,2,India,4,3,"Ores, slag and ash","14,483,115.96",1.80%,3.34%
1449,Mozambique,2021,Export,2,India,5,6,"Fruit and nuts, edible; peel of citrus fruit or melons","11,440,661.04",1.42%,9.64%


## Análise de simetrias nos dados

In [33]:
cols = ['reporterDesc','partnerDesc','refYear','flowDesc','primaryValueFormated','primaryValue']

partnerA = country_of_interest
partnerB = comtrade.COUNTRY_CODES_REVERSE['United States of America']

### Principais destinos de exportação reportados pelo país

In [35]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(20)

,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
105,Mozambique,South Africa,2020,Export,"731,131,152.99",7.311312e+08
100,Mozambique,India,2020,Export,"423,058,822.87",4.230588e+08
121,Mozambique,United Kingdom,2020,Export,"373,427,676.38",3.734277e+08
22,Mozambique,China,2020,Export,"255,169,317.16",2.551693e+08
52,Mozambique,Italy,2020,Export,"237,807,485.79",2.378075e+08
77,Mozambique,Netherlands,2020,Export,"233,481,470.79",2.334815e+08
9,Mozambique,Belgium,2020,Export,"121,389,279.45",1.213893e+08
101,Mozambique,Singapore,2020,Export,"120,906,559.86",1.209066e+08
106,Mozambique,Zimbabwe,2020,Export,"109,795,452.06",1.097955e+08
107,Mozambique,Spain,2020,Export,"106,292,608.83",1.062926e+08


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [36]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(20)

,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
185,India,Mozambique,2020,Import,"711,594,556.61",7.115946e+08
190,South Africa,Mozambique,2020,Import,"658,881,361.39",6.588814e+08
40,China,Mozambique,2020,Import,"579,498,619.00",5.794986e+08
95,Italy,Mozambique,2020,Import,"346,739,406.00",3.467394e+08
220,United Arab Emirates,Mozambique,2020,Import,"329,738,747.17",3.297387e+08
210,Spain,Mozambique,2020,Import,"239,108,272.25",2.391083e+08
154,Netherlands,Mozambique,2020,Import,"176,736,798.01",1.767368e+08
77,Germany,Mozambique,2020,Import,"154,233,342.12",1.542333e+08
162,Poland,Mozambique,2020,Import,"147,281,705.00",1.472817e+08
191,Zimbabwe,Mozambique,2020,Import,"136,611,176.77",1.366112e+08


In [20]:
flowCode = 'X'
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
print(f"Importações de produtos de {comtrade.COUNTRY_CODES.get(partnerA)} por {comtrade.COUNTRY_CODES.get(partnerB)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
df_import_rep_a[cols]

Importações de produtos de Mozambique por United States of America reportados por Mozambique


,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,Mozambique,United States of America,2021,Export,"102,338,208.05",1.023382e+08


In [24]:
# invertemos

print(years_of_interest, partnerA, partnerB)
print(f"Exportações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")

flowCode = 'M'
df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=None,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
df_import_rep_b[cols]

2021 508 842
Exportações de produtos de United States of America para Mozambique reportados por United States of America


,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,United States of America,Mozambique,2021,Import,"159,823,538.00",159823538.0
